In [18]:
import pandas as pd
import duckdb

In [17]:
# Установка библиотеки duckdb
# pip install duckdb

# Задание 1

In [15]:
# Задание таблиц БД
users = pd.read_csv('users.csv')
course_users = pd.read_csv('course_users.csv')
courses = pd.read_csv('courses.csv')
course_types = pd.read_csv('course_types.csv')
lessons = pd.read_csv('lessons.csv')
subjects = pd.read_csv('subjects.csv')
cities = pd.read_csv('cities.csv')
homework_done = pd.read_csv('homework_done.csv')
homework = pd.read_csv('homework.csv')
homework_lessons = pd.read_csv('homework_lessons.csv')
user_roles = pd.read_csv('user_roles.csv') 

In [218]:
# Задание SQL-запроса
query = """
-- Напишите здесь ваш SQL-запрос
SELECT
courses.id AS course_id,
courses.name AS courses_name,
subjects.name AS subject_name,
subjects.project AS subject_type,
course_types.name AS course_type,
courses.starts_at AS courses_date_start,
users.ID AS user_id,
users.last_name AS last_name,
cities.name AS city_name,
course_users.active,
/* не понимаю какая дата является дата_открытия_курса_ученику */
course_users.created_at AS дата_открытия_курса_ученику, 
/* получилось много отрицательных значений - либо расчет неправильный, либо
это так и надо 
P.S. вижу во втором задании, что студенты могли добавиться на курс до его старта*/
    CASE
        WHEN course_users.updated_at IS NOT NULL 
        THEN
            EXTRACT(YEAR FROM AGE(CAST(course_users.updated_at AS TIMESTAMP), CAST(course_users.created_at AS TIMESTAMP))) * 12 +
            EXTRACT(MONTH FROM AGE(CAST(course_users.updated_at AS TIMESTAMP), CAST(course_users.created_at AS TIMESTAMP)))
        ELSE
            0
    END AS full_months_course_opened_for_user,
    COUNT(DISTINCT homework_done.homework_id) AS hmwrk_cnt,
    -- в задании не указано, но нужно для построения дашборда
    course_users.updated_at
FROM
users 
LEFT JOIN cities  ON users.city_id = cities.id
LEFT JOIN course_users  ON users.id = course_users.user_id 
LEFT JOIN courses  ON course_users.course_id = courses.id 
LEFT JOIN subjects  ON courses.subject_id = subjects.id
LEFT JOIN course_types  ON courses.course_type_id = course_types.id
--LEFT JOIN homework_done  ON users.id = homework_done.user_id
LEFT JOIN lessons ON courses.id = lessons.course_id
LEFT JOIN homework_lessons ON lessons.id = homework_lessons.lesson_id
LEFT JOIN homework_done ON homework_lessons.homework_id = homework_done.homework_id
-- можно было бы посчитать оконной функцией  вместо группировки и COUNT 
GROUP BY courses.id, courses.name, subjects.name, subjects.project, course_types.name, courses.starts_at,
users.ID, users.last_name, cities.name, course_users,  course_users, full_months_course_opened_for_user,
course_users.active, course_users.created_at, course_users.updated_at
"""

# Выполнение SQL-запроса
df_result = duckdb.query(query).to_df()

# Вывод результата

display(df_result)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,course_id,courses_name,subject_name,subject_type,course_type,courses_date_start,user_id,last_name,city_name,active,дата_открытия_курса_ученику,full_months_course_opened_for_user,hmwrk_cnt,updated_at
0,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10005,Гусев,Белгород,0.0,2024-07-26 05:38:41,0,5,2024-07-26 05:38:41
1,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10006,Пушкина,Москва,0.0,2024-07-25 18:48:31,0,5,2024-07-25 18:48:31
2,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10009,Чернобель,Минеральные Воды,0.0,2024-07-26 06:05:10,0,5,2024-07-26 06:05:10
3,50.0,Скорая помощь по русскому 2к24,Русский,ЕГЭ,Спецкурс,2024-10-24,10024,Имамутдинова,Туймазы,0.0,2024-09-14 15:27:21,0,2,2024-09-14 15:27:21
4,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10036,Цветаев,None,0.0,2024-07-26 14:56:55,0,5,2024-07-26 14:56:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16753,NaN,None,None,None,None,None,30905,Арбузова,Тула,NaN,None,0,0,None
16754,NaN,None,None,None,None,None,33052,Ерогова,Пермь,NaN,None,0,0,None
16755,NaN,None,None,None,None,None,30565,Маслов,None,NaN,None,0,0,None
16756,NaN,None,None,None,None,None,30875,Газарян,None,NaN,None,0,0,None


# Задание 2 

In [219]:
# количество строк истолбцов в датасете
df_result.shape

(16758, 14)

In [220]:
df_result.head(5)

,course_id,courses_name,subject_name,subject_type,course_type,courses_date_start,user_id,last_name,city_name,active,дата_открытия_курса_ученику,full_months_course_opened_for_user,hmwrk_cnt,updated_at
0,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10005,Гусев,Белгород,0.0,2024-07-26 05:38:41,0,5,2024-07-26 05:38:41
1,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10006,Пушкина,Москва,0.0,2024-07-25 18:48:31,0,5,2024-07-25 18:48:31
2,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10009,Чернобель,Минеральные Воды,0.0,2024-07-26 06:05:10,0,5,2024-07-26 06:05:10
3,50.0,Скорая помощь по русскому 2к24,Русский,ЕГЭ,Спецкурс,2024-10-24,10024,Имамутдинова,Туймазы,0.0,2024-09-14 15:27:21,0,2,2024-09-14 15:27:21
4,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10036,Цветаев,None,0.0,2024-07-26 14:56:55,0,5,2024-07-26 14:56:55


In [221]:
# так можно посмотреть описательную статистику по количественным переменным в датасете, в 
# max и min можно увидеть выбросы  (outliers)
df_result.describe()

,course_id,user_id,active,full_months_course_opened_for_user,hmwrk_cnt
count,15347.000000,16758.000000,15347.000000,16758.000000,16758.000000
mean,51.842771,23180.497136,0.779762,0.152285,7.371047
std,32.596828,8927.906999,0.414421,0.514710,9.908445
min,16.000000,10001.000000,0.000000,0.000000,0.000000
25%,22.000000,10800.000000,1.000000,0.000000,0.000000
50%,41.000000,25721.000000,1.000000,0.000000,1.000000
75%,88.000000,30276.000000,1.000000,0.000000,15.000000
max,104.000000,33999.000000,1.000000,3.000000,35.000000


In [222]:
# дублей нет
df_result[df_result.duplicated()]

,course_id,courses_name,subject_name,subject_type,course_type,courses_date_start,user_id,last_name,city_name,active,дата_открытия_курса_ученику,full_months_course_opened_for_user,hmwrk_cnt,updated_at


In [223]:
# Количество пользователей, у которых нет курсов
df_result.query('courses_name.isnull()') \
        .groupby('user_id') \
        .agg({'user_id':'nunique'}) \
        .sum()

user_id    1411
dtype: int64

In [224]:
# Количество пользователей, у которых есть курсы
df_result.query('courses_name.notnull()') \
        .groupby('user_id') \
        .agg({'user_id':'nunique'}) \
        .sum()

user_id    3495
dtype: int64

In [225]:
# смотрю типы данных в столбцах, вижу, что даты надо перевести в формат datetime
df_result.dtypes

course_id                             float64
courses_name                           object
subject_name                           object
subject_type                           object
course_type                            object
courses_date_start                     object
user_id                                 int64
last_name                              object
city_name                              object
active                                float64
дата_открытия_курса_ученику            object
full_months_course_opened_for_user      int64
hmwrk_cnt                               int64
updated_at                             object
dtype: object

In [226]:
# перевожу переменные в формат dateime
df_result['courses_date_start'] = pd.to_datetime(df_result['courses_date_start'])
df_result['дата_открытия_курса_ученику'] = pd.to_datetime(df_result['дата_открытия_курса_ученику'])

In [227]:
# создаю переменную с количеством дней с разницей между датой открытия ученику и датой старта курса
df_result['days_diff'] = (df_result['дата_открытия_курса_ученику'] - df_result['courses_date_start']).dt.days

# создаю фугкцию для определения волны по неделе присоединения
def assign_wave(days_diff):
    if days_diff <= 0:
        return 0  # 0 волна
    elif days_diff <= 7:
        return 1  # 1 волна
    elif days_diff <= 14:
        return 2  # 2 волна
    elif days_diff <= 21:
        return 3  # 3 волна
    elif days_diff <= 28:
        return 4  # 4 волна
    elif days_diff > 28:
        return 5  # 5 волна
    else:
        return 6  # 6 волна, сюда попадут ученики без курсов

# применяю функцию, создаю столбец wave
df_result['wave'] = df_result['days_diff'].apply(assign_wave)

In [228]:
# сразу вижу, что у ученик может присоединиться за 44 дня до старта курса. Возникают вопросы.
# либо я где-то ошибся (а у меня были сомнения в выборе полей в таблицах) или так может быть и надо уточнить
# у продактов или это артефакты, ошибки в данных
df_result.head(5)

,course_id,courses_name,subject_name,subject_type,course_type,courses_date_start,user_id,last_name,city_name,active,дата_открытия_курса_ученику,full_months_course_opened_for_user,hmwrk_cnt,updated_at,days_diff,wave
0,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10005,Гусев,Белгород,0.0,2024-07-26 05:38:41,0,5,2024-07-26 05:38:41,38.0,5
1,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10006,Пушкина,Москва,0.0,2024-07-25 18:48:31,0,5,2024-07-25 18:48:31,37.0,5
2,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10009,Чернобель,Минеральные Воды,0.0,2024-07-26 06:05:10,0,5,2024-07-26 06:05:10,38.0,5
3,50.0,Скорая помощь по русскому 2к24,Русский,ЕГЭ,Спецкурс,2024-10-24,10024,Имамутдинова,Туймазы,0.0,2024-09-14 15:27:21,0,2,2024-09-14 15:27:21,-40.0,0
4,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10036,Цветаев,None,0.0,2024-07-26 14:56:55,0,5,2024-07-26 14:56:55,38.0,5


In [229]:
# посмотрим на статистику новой переменной
df_result['days_diff'].describe()
# вижу, что по этим данным ученик мог присоединиться за год до старта курса, что весьма сомнительно


count    15347.000000
mean         0.966834
std         25.643424
min       -367.000000
25%         -1.000000
50%          1.000000
75%         15.000000
max        137.000000
Name: days_diff, dtype: float64

In [230]:
# посчитаю сколько есть случаев
more_than_30_days = df_result.query('days_diff > 30').shape[0]
more_than_60_days = df_result.query('days_diff > 60').shape[0]
more_than_90_days = df_result.query('days_diff > 90').shape[0]
less_than_30_days = df_result.query('days_diff < 30').shape[0]

# Форматируем вывод
(
    f'Когда ученик присоединился к курсу более чем за месяц до старта: {more_than_30_days}',
        f'Когда ученик присоединился к курсу более чем за 2 месяца до старта: {more_than_60_days}',
        f'Когда ученик присоединился к курсу более чем за 3 месяца до старта: {more_than_90_days}',
    f'И случаи, когда меньше месяца: {less_than_30_days}'
)

('Когда ученик присоединился к курсу более чем за месяц до старта: 1742',
 'Когда ученик присоединился к курсу более чем за 2 месяца до старта: 99',
 'Когда ученик присоединился к курсу более чем за 3 месяца до старта: 6',
 'И случаи, когда меньше месяца: 13599')

Я не знаю почему так много таких случаев - надо уточнять возможно ли это

In [231]:
# Сделаю предположение, что возможно, 
# смотрю количество учеников по волнам

df_result.groupby('wave') \
        .agg({'wave':'count'}) \
        .rename(columns = {'user_id':'number'})

,wave
wave,
0,7543
1,2536
2,1363
3,1074
4,1066
5,1765
6,1411


In [232]:
# Небольшая проверка и напоминание, что у одного ученика может быть несколько курсов и предметов
df_result.groupby('wave').agg({'user_id':'count'}).sum()

user_id    16758
dtype: int64

In [233]:
# вижу, что курс может содержать несколько предметов, которые могут быть открыты в разное время ил же в одно время
# возможно я использовал неправильное поле - надо уточнить
df_result.sort_values(['last_name', 'courses_name']).tail(60)

,course_id,courses_name,subject_name,subject_type,course_type,courses_date_start,user_id,last_name,city_name,active,дата_открытия_курса_ученику,full_months_course_opened_for_user,hmwrk_cnt,updated_at,days_diff,wave
9703,21.0,Годовой 2к25 стандарт,Химия,ЕГЭ,Годовой,2024-09-16,10545,Ярбилова,Махачкала,1.0,2024-07-31 19:10:56,0,20,2024-07-31 19:10:56,-47.0,0
9936,25.0,Годовой 2к25 стандарт,Биология,ЕГЭ,Годовой,2024-09-17,10545,Ярбилова,Махачкала,1.0,2024-07-31 19:10:56,2,34,2024-10-01 22:02:11,-48.0,0
15152,44.0,КОНСПЕКТЫ ПО ЗООЛОГИИ,Биология,ЕГЭ,Спецкурс,2024-08-19,10545,Ярбилова,Махачкала,1.0,2024-08-29 16:31:52,0,0,2024-08-29 16:31:52,10.0,2
9498,102.0,"Курс ""Кульный"" по итоговому сочинению 2к24",Русский,ЕГЭ,Спецкурс,2024-11-04,10545,Ярбилова,Махачкала,1.0,2024-11-03 14:45:03,0,0,2024-11-03 14:45:03,-1.0,0
904,29.0,Летняя школа 2к24,Русский,ЕГЭ,Спецкурс,2024-06-21,10545,Ярбилова,Махачкала,0.0,2024-07-31 19:10:56,0,0,2024-07-31 19:10:56,40.0,5
1402,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10545,Ярбилова,Махачкала,0.0,2024-07-31 19:10:56,0,5,2024-07-31 19:10:56,43.0,5
1109,40.0,НЕдетский курс,Обществознание,ЕГЭ,Спецкурс,2024-08-04,10545,Ярбилова,Махачкала,1.0,2024-08-05 12:14:09,0,0,2024-08-05 12:14:09,1.0,1
9497,43.0,Химическое равновесие,Химия,ЕГЭ,Спецкурс,2024-08-23,10545,Ярбилова,Махачкала,1.0,2024-08-29 16:31:46,0,2,2024-08-29 16:31:46,6.0,1
8239,NaN,None,None,None,None,NaT,25244,Яризов,Красноярск,NaN,NaT,0,0,None,NaN,6
4401,89.0,Вычисляй как Бог,Математика,ЕГЭ,Спецкурс,2024-10-26,30164,Ярова,Нижний Новгород,1.0,2024-10-28 11:23:36,0,0,2024-10-28 11:23:36,2.0,1


In [235]:
df_result

,course_id,courses_name,subject_name,subject_type,course_type,courses_date_start,user_id,last_name,city_name,active,дата_открытия_курса_ученику,full_months_course_opened_for_user,hmwrk_cnt,updated_at,days_diff,wave
0,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10005,Гусев,Белгород,0.0,2024-07-26 05:38:41,0,5,2024-07-26 05:38:41,38.0,5
1,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10006,Пушкина,Москва,0.0,2024-07-25 18:48:31,0,5,2024-07-25 18:48:31,37.0,5
2,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10009,Чернобель,Минеральные Воды,0.0,2024-07-26 06:05:10,0,5,2024-07-26 06:05:10,38.0,5
3,50.0,Скорая помощь по русскому 2к24,Русский,ЕГЭ,Спецкурс,2024-10-24,10024,Имамутдинова,Туймазы,0.0,2024-09-14 15:27:21,0,2,2024-09-14 15:27:21,-40.0,0
4,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10036,Цветаев,None,0.0,2024-07-26 14:56:55,0,5,2024-07-26 14:56:55,38.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16753,NaN,None,None,None,None,NaT,30905,Арбузова,Тула,NaN,NaT,0,0,None,NaN,6
16754,NaN,None,None,None,None,NaT,33052,Ерогова,Пермь,NaN,NaT,0,0,None,NaN,6
16755,NaN,None,None,None,None,NaT,30565,Маслов,None,NaN,NaT,0,0,None,NaN,6
16756,NaN,None,None,None,None,NaT,30875,Газарян,None,NaN,NaT,0,0,None,NaN,6


In [255]:
df_result['updated_at'] = pd.to_datetime(df_result['updated_at'])
df_result['signup_month'] = df_result['дата_открытия_курса_ученику'].dt.to_period('M')
df_result['activity_month'] = df_result['updated_at'].dt.to_period('M')

In [256]:
df_result['activity_month']

0        2024-07
1        2024-07
2        2024-07
3        2024-09
4        2024-07
          ...   
16753        NaT
16754        NaT
16755        NaT
16756        NaT
16757        NaT
Name: activity_month, Length: 16758, dtype: period[M]

In [308]:
df_result['month_diff'] = (df_result['updated_at'].dt.year - df_result['дата_открытия_курса_ученику'].dt.year) * 12 + (df_result['updated_at'].dt.month - df_result['дата_открытия_курса_ученику'].dt.month)
 

In [309]:
# группировка для сводной таблицы
for_pivot = df_result.query("active == 1") \
    .groupby(['course_id', 'courses_name', 'month_diff'], as_index = False) \
    .agg({'user_id':'count'}) \
    .rename(columns = {'user_id' : 'count'})
for_pivot

,course_id,courses_name,month_diff,count
0,16.0,Годовой 2к25 стандарт,0.0,718
1,16.0,Годовой 2к25 стандарт,1.0,588
2,16.0,Годовой 2к25 стандарт,2.0,175
3,16.0,Годовой 2к25 стандарт,3.0,144
4,16.0,Годовой 2к25 стандарт,4.0,36
...,...,...,...,...
117,100.0,"Курс ""Кульный"" по итоговому сочинению 2к24 с п...",0.0,103
118,101.0,"Курс ""Кульный"" по итоговому сочинению 2к24 для...",0.0,922
119,102.0,"Курс ""Кульный"" по итоговому сочинению 2к24",0.0,2125
120,103.0,Погоня за мечтой 2к25,0.0,5


In [310]:
# Сводная таблица - retention
pivot_table = for_pivot.pivot_table(
    index=['course_id', 'courses_name'],          
    columns='month_diff',  
    values='count',                               
    aggfunc='sum',                                
    fill_value=0                                  
)
pivot_table

,month_diff,0.0,1.0,2.0,3.0,4.0
course_id,courses_name,,,,,
16.0,Годовой 2к25 стандарт,718,588,175,144,36
17.0,Годовой 2к25 стандарт,174,144,48,36,4
18.0,Годовой 2к25 стандарт,146,132,35,29,7
19.0,Годовой 2к25 стандарт,392,298,85,85,11
20.0,Годовой 2к25 стандарт,61,36,11,4,1
21.0,Годовой 2к25 стандарт,70,38,18,11,7
22.0,Годовой 2к25 стандарт,171,178,49,38,5
23.0,Годовой 2К25 стандарт с Машей,149,133,40,31,6
24.0,В погоне за пятеркой 2К25 стандарт,100,42,8,5,1


In [268]:
# выгрузил для 3 задания
df_result.to_csv('df_result.csv', index=False, encoding='utf-8-sig')

In [311]:
df_result

,course_id,courses_name,subject_name,subject_type,course_type,courses_date_start,user_id,last_name,city_name,active,дата_открытия_курса_ученику,full_months_course_opened_for_user,hmwrk_cnt,updated_at,days_diff,wave,signup_month,activity_month,month_diff
0,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10005,Гусев,Белгород,0.0,2024-07-26 05:38:41,0,5,2024-07-26 05:38:41,38.0,5,2024-07,2024-07,0.0
1,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10006,Пушкина,Москва,0.0,2024-07-25 18:48:31,0,5,2024-07-25 18:48:31,37.0,5,2024-07,2024-07,0.0
2,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10009,Чернобель,Минеральные Воды,0.0,2024-07-26 06:05:10,0,5,2024-07-26 06:05:10,38.0,5,2024-07,2024-07,0.0
3,50.0,Скорая помощь по русскому 2к24,Русский,ЕГЭ,Спецкурс,2024-10-24,10024,Имамутдинова,Туймазы,0.0,2024-09-14 15:27:21,0,2,2024-09-14 15:27:21,-40.0,0,2024-09,2024-09,0.0
4,33.0,Летняя школа 2к24,Математика,ЕГЭ,Спецкурс,2024-06-18,10036,Цветаев,None,0.0,2024-07-26 14:56:55,0,5,2024-07-26 14:56:55,38.0,5,2024-07,2024-07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16753,NaN,None,None,None,None,NaT,30905,Арбузова,Тула,NaN,NaT,0,0,NaT,NaN,6,NaT,NaT,NaN
16754,NaN,None,None,None,None,NaT,33052,Ерогова,Пермь,NaN,NaT,0,0,NaT,NaN,6,NaT,NaT,NaN
16755,NaN,None,None,None,None,NaT,30565,Маслов,None,NaN,NaT,0,0,NaT,NaN,6,NaT,NaT,NaN
16756,NaN,None,None,None,None,NaT,30875,Газарян,None,NaN,NaT,0,0,NaT,NaN,6,NaT,NaT,NaN


In [312]:
perc_table = pivot_table.reset_index()
#perc_table

In [313]:
perc_table['first_mp'] = 100
perc_table['second_mp'] = round((perc_table.iloc[:, 3] / perc_table.iloc[:, 2] * 100),0).astype(int)
perc_table['third_mp'] = (perc_table.iloc[:, 4] / perc_table.iloc[:, 2]  * 100).astype(int)
perc_table['fourth_mp'] = (perc_table.iloc[:, 5] / perc_table.iloc[:, 2]  * 100).astype(int)
perc_table['fifth_mp'] = (perc_table.iloc[:, 6] / perc_table.iloc[:, 2]  * 100).astype(int)

In [314]:
perc_table = perc_table.drop(perc_table.columns[2:7], axis=1)
#perc_table

In [315]:
perc_table['first_mp'] = perc_table['first_mp'].astype(str) + '%'
perc_table['second_mp'] = perc_table['second_mp'].astype(str) + '%'
perc_table['third_mp'] = perc_table['third_mp'].astype(str) + '%'
perc_table['fourth_mp'] = perc_table['fourth_mp'].astype(str) + '%'
perc_table['fifth_mp'] = perc_table['fifth_mp'].astype(str) + '%'

In [316]:
perc_table
# вижу, что retention в некоторые месяцы по некоторым курсам 
# больше 100% - это невозможно, надо выяснять причину

month_diff,course_id,courses_name,first_mp,second_mp,third_mp,fourth_mp,fifth_mp
0,16.0,Годовой 2к25 стандарт,100%,82%,24%,20%,5%
1,17.0,Годовой 2к25 стандарт,100%,83%,27%,20%,2%
2,18.0,Годовой 2к25 стандарт,100%,90%,23%,19%,4%
3,19.0,Годовой 2к25 стандарт,100%,76%,21%,21%,2%
4,20.0,Годовой 2к25 стандарт,100%,59%,18%,6%,1%
5,21.0,Годовой 2к25 стандарт,100%,54%,25%,15%,10%
6,22.0,Годовой 2к25 стандарт,100%,104%,28%,22%,2%
7,23.0,Годовой 2К25 стандарт с Машей,100%,89%,26%,20%,4%
8,24.0,В погоне за пятеркой 2К25 стандарт,100%,42%,8%,5%,1%
9,25.0,Годовой 2к25 стандарт,100%,122%,31%,30%,3%


In [317]:
# Не разобрался как сделать таблицу с процентами по первому столбцу в Datalens - сделал здесь
perc_table.to_csv('perc_table.csv', index=False, encoding='utf-8-sig')
